<a href="https://colab.research.google.com/github/nabihsabeh85/Chatbot-/blob/main/Chat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Setting up Environment Variables

In [3]:
import os

os.environ["OPENAI_API_KEY"] = ""
os.environ["PINECONE_API_KEY"] = ""
os.environ["PINECONE_ENVIRONMENT"] = ""
os.environ["PINECONE_INDEX_NAME"] = ""
os.environ["PINECONE_KNOWLEDGE_NAMESPACE"] = ""
os.environ["PINECONE_CHAT_NAMESPACE"] = ""
os.environ["UPSTASH_REDIS_REST_URL"] = ""
os.environ["UPSTASH_REDIS_REST_TOKEN"] = ""


In [ ]:
#Installing Required Packages
!pip install aiohttp aiosignal annotated-types anyio attrs certifi charset-normalizer click colorama dataclasses-json distro dnspython email_validator fastapi fastapi-cli frozenlist greenlet h11 httpcore httptools httpx idna Jinja2 jsonpatch jsonpointer langchain langchain-community langchain-core langchain-openai langchain-pinecone langchain-text-splitters langsmith markdown-it-py MarkupSafe marshmallow mdurl multidict mypy-extensions numpy openai orjson packaging pinecone pinecone-client pydantic pydantic_core Pygments python-dotenv python-multipart PyYAML regex requests rich shellingham six sniffio SQLAlchemy starlette tenacity tiktoken tqdm typer typing-inspect typing_extensions ujson upstash-redis urllib3 uvicorn watchfiles websockets yarl


In [ ]:
#test_assistant.py
import urllib3
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import ChatOpenAI
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain_pinecone import PineconeVectorStore
import datetime
import os

CONDENSE_TEMPLATE = """Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question.
<chat_history>
  {chat_history}
</chat_history>
Follow Up Input: {question}
Standalone question:"""

QA_TEMPLATE = """You are an expert report analyst. Use the following pieces of context to answer the question at the end.
Current time: {current_time}.
Please answer the question considering the context.
<context>
  {context}
</context>
<chat_history>
  {chat_history}
</chat_history>
Question: {question}
Answer:
"""

class EducatorAssistant:
    def __init__(self, pinecone_index):
        self.model = ChatOpenAI(temperature=0, model="gpt-4o")
        self.index_name = os.getenv("PINECONE_INDEX_NAME")
        self.embed_model = OpenAIEmbeddings(
            model="text-embedding-ada-002",
            openai_api_key=os.getenv("OPENAI_API_KEY")
        )
        self.history = []

        self.pinecone_index = pinecone_index
        self.prompt_qa = ChatPromptTemplate.from_messages([
            ("system", QA_TEMPLATE),
        ])
        self.prompt_condense = ChatPromptTemplate.from_messages([
            ("system", CONDENSE_TEMPLATE),
        ])
        self.chain_qa = self.prompt_qa | self.model
        self.chain_condense = self.prompt_condense | self.model

    def chat(self, user_input):
        condense_question = self.chain_condense.invoke({
            "question": user_input,
            "chat_history": self.history
        })

        vectorstore_knowledge = PineconeVectorStore(
            index_name=self.index_name, embedding=self.embed_model, namespace=os.getenv("PINECONE_KNOWLEDGE_NAMESPACE")
        )

        query = condense_question.content

        docs_knowledge = vectorstore_knowledge.similarity_search(query, k=20)
        separator = '\n\n'
        serialized_docs_knowledge = [doc.page_content for doc in docs_knowledge]
        knowledge_context = separator.join(serialized_docs_knowledge)

        separator = '\n'
        serialized_history = [f"{key}: {value}" for message in self.history[-5:] for key, value in message.items()]
        history = separator.join(serialized_history)

        current_datetime = datetime.datetime.now()
        current_datetime_str = current_datetime.strftime("%Y-%m-%d %H:%M:%S")

        answer = self.chain_qa.invoke({
            "question": user_input,
            "chat_history": history,
            "context": knowledge_context,
            "current_time": current_datetime_str
        })
        self.history.append({"Human": user_input})
        self.history.append({"Assistant": answer.content})
        return answer.content


In [4]:
#test_server.py
from dotenv import load_dotenv
from upstash_redis import Redis
import uvicorn
from fastapi import FastAPI, Query
import os
import uuid
import json
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from pinecone import Pinecone
from pydantic import BaseModel
from test_assistant import EducatorAssistant
from fastapi.middleware.cors import CORSMiddleware

load_dotenv()

upstash_redis_url = os.getenv("UPSTASH_REDIS_REST_URL")
upstash_redis_token = os.getenv("UPSTASH_REDIS_REST_TOKEN")
pinecone_api_key = os.getenv("PINECONE_API_KEY")
pinecone_index_name = os.getenv("PINECONE_INDEX_NAME")
openai_api_key = os.getenv("OPENAI_API_KEY")

redis = Redis(
    url=upstash_redis_url,
    token=upstash_redis_token
)

pc = Pinecone(api_key=pinecone_api_key)
index = pc.Index(pinecone_index_name)

app = FastAPI()

origins = ["*"]

app.add_middleware(
    CORSMiddleware,
    allow_origins=origins,
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

sessions = {}
from typing import Dict
from datetime import datetime, timedelta

client_last_activity: Dict[str, datetime] = {}

def generate_unique_uuid():
    return str(uuid.uuid1())

def set_messages(user_id, messages):
    res = redis.set(user_id, {"data": messages})
    return res

def get_messages(user_id):
    messages = redis.get(user_id)
    if messages is None:
        return []
    return json.loads(messages)["data"]

class MessageList(BaseModel):
    session_id: str
    message: str

@app.post("/start")
async def say_hello():
    session_id = generate_unique_uuid()
    assistant = EducatorAssistant(index)
    sessions[session_id] = assistant
    return {
        "session_id": session_id,
        "message": "Hello, How can I help you?"
    }

@app.post("/end")
async def end(req: MessageList):
    assistant = None
    if req.session_id in sessions:
        assistant = sessions[req.session_id]
        result = set_messages(req.session_id, assistant.history)
        sessions.pop(req.session_id)
        if result:
            return "save success"
        else:
            return "save failed"
    else:
        return "not found"

@app.post("/chat")
async def chat_with_teacher_agent(req: MessageList):
    assistant = None
    if req.session_id in sessions:
        assistant = sessions[req.session_id]
        client_last_activity[req.session_id] = datetime.utcnow()
    else:
        assistant = EducatorAssistant(index)
        sessions[req.session_id] = assistant
    response = assistant.chat(req.message)
    return {"message": response}

async def cleanup_chat_histories():
    current_time = datetime.utcnow()
    for session_id, last_activity in list(client_last_activity.items()):
        if current_time - last_activity > timedelta(seconds=1800):
            message_list = MessageList(session_id=session_id, message='end')
            await end(message_list)
            del client_last_activity[session_id]

async def repeat_cleanup_task(wait_time: int):
    while True:
        await cleanup_chat_histories()
        await asyncio.sleep(wait_time)

@app.on_event("startup")
async def on_startup():
    asyncio.create_task(repeat_cleanup_task(1800))

if __name__ == "__main__":
    uvicorn.run("test_server:app", host="0.0.0.0", port=8001, reload=True)
